In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from skimage.morphology import skeletonize

# Paths
base_path = "/content/drive/MyDrive/Autism Images"
transformed_path = os.path.join("/content/drive/MyDrive/", "TransformedImages")
folders = ["TCImages", "TSImages"]

# Create folders for transformed images
for folder in folders:
    os.makedirs(os.path.join(transformed_path, folder), exist_ok=True)

# Processing function
def preprocess_image(img_path):
    # Load image
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Thresholding (binary inverse if lines are dark)
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

    # CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    clahe_img = clahe.apply(thresh)

    # Canny Edge Detection
    edges = cv2.Canny(clahe_img, threshold1=50, threshold2=150)

    # Dilation to connect edges
    kernel = np.ones((3,3), np.uint8)
    dilated = cv2.dilate(edges, kernel, iterations=1)

    # Skeletonization (convert to 0-1 first)
    skeleton = skeletonize(dilated // 255)
    skeleton_img = (skeleton * 255).astype(np.uint8)

    # Resize and normalize
    final_img = cv2.resize(skeleton_img, (224, 224))

    return final_img

# Loop through folders and images
for folder in folders:
    input_folder = os.path.join(base_path, folder)
    output_folder = os.path.join(transformed_path, folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(input_folder, filename)
            transformed_img = preprocess_image(img_path)

            save_path = os.path.join(output_folder, filename)
            cv2.imwrite(save_path, transformed_img)

print("All images transformed and saved successfully!")


All images transformed and saved successfully!
